<a href="https://colab.research.google.com/github/Mitchell7777/Mitchell-s-physics-stuff/blob/master/11_wave_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wave Equation
# Mitchell Wright
The electromagnetic fields in empty space obey a wave equation:
\begin{align*}
\dfrac{\partial^2 \vec{E}}{\partial t^2} - c^2 \nabla^2 \vec{E} &= 0 \\
\dfrac{\partial^2 \vec{B}}{\partial t^2} - c^2 \nabla^2 \vec{B} &= 0 \\
\end{align*}

The scalar and vector potentials in the Lorentz gauge obey an inhomogenous wave equation as well:
\begin{align*}
\dfrac{1}{c^2} \dfrac{\partial^2 \Phi}{\partial t^2} - \nabla^2 \Phi &= 4\pi \rho \\
\dfrac{1}{c^2} \dfrac{\partial^2 \vec{A}}{\partial t^2} - \nabla^2 \vec{A} &= \dfrac{4\pi}{c^2} \vec{J} \\
\end{align*}

This notebook will explore numerical methods for solving the wave equation.  We will explore specific examples in electrodynamics in the coming weeks.

The general form of the wave equation is
$$\dfrac{1}{c^2} \dfrac{\partial^2 \phi}{\partial t^2} - \nabla^2 \phi = 0$$

The simulation below includes a diffusion term, too.
$$\dfrac{1}{c^2} \dfrac{\partial^2 \phi}{\partial t^2} + \dfrac{1}{D} \dfrac{\partial \phi}{\partial t} - \nabla^2 \phi = 0$$

# Installation

In [1]:
from IPython import display

In [2]:
# Need to upgrade matplotlib for some PyVista plotting commands to work.
# Run this cell, then select "Runtime > Restart runtime" from the CoLab menu.
!pip install --upgrade matplotlib

display.clear_output()

In [3]:
# This simulation uses real values.
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-complex.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

display.clear_output()

In [4]:
# Everything should be installed now.
# Import the rest of what we need.
from dolfinx import fem, mesh, plot, la
import ufl

from mpi4py import MPI
import petsc4py.PETSc as petsc

import numpy as np
import pyvista

# Start a virtual plot window for PyVista in CoLab.
pyvista.start_xvfb()
pyvista.set_jupyter_backend("pythreejs")

# Get tools for embedding movies in CoLab.
from base64 import b64encode

In [5]:
if not np.issubdtype(petsc.ScalarType, np.complexfloating):
    print("This tutorial requires complex number support")
    sys.exit(0)
else:
    print(f"Using {petsc.ScalarType}.")

Using <class 'numpy.complex128'>.


# Real Time Simulation

One approach to studying the wave equation is to use a finite difference method.  We approximate the time derivative as follows:
$$
\dfrac{\partial^2 \phi}{\partial t^2}
\approx  
\dfrac{\phi_{n+1} - 2 \phi_{n} + \phi_{n-1}}{(\Delta t)^2}
$$
We then adapt the finite element problem to include a contribution from the two preceding time steps and solve for $\phi_{n+1}$ over and over.

Let's return to the potential from the previous notebook on the diffusion equation and observe the impact of a seemingly minor change in the code.

## Model

The model is a gaussian charge distribution inside of a box with grounded walls.

You can change the boundary conditions to explore other systems.

### Geometry

In [6]:
# Create a simple rectangular mesh.
length = 10
height = 10
Nx, Ny = 101, 101
extent = [[-length/2, -height/2], [length/2, height/2]]
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, extent, [Nx, Ny], mesh.CellType.triangle)

In [7]:
# Prepare the mesh for plotting.
topology, cells, geometry = dolfinx.plot.create_vtk_mesh(domain)

# Turn the mesh into a PyVista grid.
grid = pyvista.UnstructuredGrid(topology, cells, geometry)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# Save the HTML file.
plotter.export_html("./grid.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/grid.html')

### Charge Density

This time, we will define the charge density as a time-dependent function on the grid.

In [10]:
# Define a set of functions on our mesh.
V = dolfinx.fem.FunctionSpace(domain, ("Lagrange", 2))
# V = dolfinx.fem.FunctionSpace(domain, ("CG", 1))

In [11]:
# Define the charge.
xC = -1
yC = -1
Q = 1
sQ = 0.2
beta = 1/sQ
tau = 1

# This function returns a function that can be interpolated.
def rho_function(t):
    return lambda x: Q * beta / 2 / np.pi * np.exp(-t/tau) * np.exp(-0.5 * beta**2 * ((x[0]-xC)**2 + (x[1] - yC)**2))

# Turn this function definition into a time-dependent function on the mesh.
rho = fem.Function(V)

# Initialize the function for t=0.
rho.interpolate(rho_function(0))

In [12]:
# Plot the initial charge density.
topology, cells, geometry = plot.create_vtk_mesh(rho.function_space)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["rho"] = rho.x.array.real
grid.set_active_scalars("rho")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
warped = grid.warp_by_scalar("rho", factor=10)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Charge Density"},
    clim=[0, 1],
    cmap='turbo'
)
plotter.add_text("Initial Charge Density")
# Save the HTML file.
plotter.export_html("./rho-initial.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/rho-initial.html')

ImportError: ignored

### Boundary Conditions

We will set the potential to zero on the boundary to compute the initial potential and fields.  After this, the boundaries will be open.

In [13]:
# Define a function to locate boundaries.
# It will return True for points on the boundary, and False otherwise.
def find_facets(x):
    yes = np.isclose(x[0], -length/2)
    yes += np.isclose(x[0], length/2)
    yes += np.isclose(x[1], -height/2)
    yes += np.isclose(x[1], height/2)
    return yes

# Next, we use this function to tag each cell along the boundary.
tdim = domain.topology.dim
bc_facets = mesh.locate_entities_boundary(
    domain, tdim - 1, find_facets)

# Identify those for which the function was True as
# boundary_dofs = "boundary degrees of freedom"
boundary_dofs = fem.locate_dofs_topological(V, tdim - 1, bc_facets)

# # Now introduce the boundary condition: constant potential on the boundary.
# Define the potential on the boundary.
def uD_function(t):
    return lambda x: 0j * x[0]

# Turn this function definition into a time-dependent function on the mesh.
uD = fem.Function(V)

# Initialize the function for t=0.
uD.interpolate(uD_function(0.0))

# Add these to the list of boundary conditions to be imposed.
# This will apply the uD function to the cells along the boundary.
# It will be updated during each time step.
bcs = [fem.dirichletbc(uD, boundary_dofs)]

### Initial Potential

In [14]:
# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
phi = fem.Function(V)
phi_n = fem.Function(V)

# This is the FEM version of the Laplacian.
# It is the left-hand side of Poisson's equation.
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

# This is how we introduce the charge density.
# It is the right-hand side of Poisson's equation.
L = 4 * ufl.pi * ufl.inner(rho,v) * ufl.dx

# Put it all together for FEniCSx.
problem = fem.petsc.LinearProblem(a, L, bcs=bcs, u=phi, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# Now, solve it!
problem.solve()

# Tie up some loose ends.
phi.vector.ghostUpdate(addv=petsc.InsertMode.INSERT, mode=petsc.ScatterMode.FORWARD)

display.clear_output()

In [16]:
# Plot the initial potential.
topology, cells, geometry = plot.create_vtk_mesh(rho.function_space)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["phi"] = phi.x.array.real
grid.set_active_scalars("phi")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
warped = grid.warp_by_scalar("phi", factor=10)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Potential"},
    clim=[0, 1],
    cmap='turbo'
)
plotter.add_text("Initial Potential")
# Save the HTML file.
plotter.export_html("./phi-initial.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/phi-initial.html')

ImportError: ignored

In [17]:
# Store this solution for reuse later.
phi_initial = phi.copy()

## Time Evolution

Let's compare the diffusion and wave equations side by side.

In [18]:
# Initial time and maximum time.
t0 = 0.0
t_max = 0.05
t_step = 0.002

# Wave speed.
c_val = 500.0

# Diffusion constant.
d_val = 200.0

# This is the time step for the simulation.
c = fem.Constant(domain, petsc.ScalarType(c_val))
D = fem.Constant(domain, petsc.ScalarType(d_val))
dt = fem.Constant(domain, petsc.ScalarType(t_step))
f = fem.Constant(domain, petsc.ScalarType(0.0))

### Diffusion

In [19]:
# Reset the initial potential.
phi = phi_initial.copy()
phi_n = fem.Function(V)

In [20]:
# This is the problem we are going to solve:
# The diffusion equaiton, in weak form for finite elements.
F = ufl.inner(u - phi_n, v) * ufl.dx
F += dt * D * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
F -= dt * rho * ufl.conj(v) * ufl.dx
(a, L) = ufl.system(F)

In [21]:
## Choose boundary conditions.
bcs = [fem.dirichletbc(uD, boundary_dofs)]
# bcs = []

In [22]:
# Building the matrix for the finite element problem takes a while.
# It does not change throughout the problem, either.  The following
# line build the matrix once and store it for reuse.
compiled_a = fem.form(a)
A = fem.petsc.assemble_matrix(compiled_a, bcs=bcs)
A.assemble()

display.clear_output()

In [23]:
# We do the same for the vector, but it will be updated at each time step.
compiled_L = fem.form(L)
b = fem.Function(V)

display.clear_output()

In [24]:
# Now we use the PETSc problem to construct the linear algebra problem.
solver = petsc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(petsc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(petsc.PC.Type.LU)

In [25]:
# Set potential to initial solution.
phi = phi_initial.copy()

# Create a plotter for charge density, but don't display the plot.
phi_file = "%03d-diffusion.jpg"

# It will be making a movie in the background.
phi_plotter = pyvista.Plotter(notebook=False, off_screen=True)

# And add the charge and potentials to the plot.
topology, cells, geometry = plot.create_vtk_mesh(phi.function_space)
phi_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
phi_grid.point_data["phi"] = phi.x.array.real
phi_grid.set_active_scalars("phi")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
phi_warped = phi_grid.warp_by_scalar("phi", factor=10)

# Add the warped grid to the plot.
phi_plotter.add_mesh(
    phi_warped,
    lighting=False,
    show_edges=False,
    clim=[0, 1],
    cmap='turbo'
);

ImportError: ignored

In [ ]:
# Get rid of old diffusion movie files.
!rm *-diffusion.jpg
!rm diffusion-movie.mp4

In [ ]:
# Now, compute an update for each time step and add it to the movie.
# Initial time
t = t0
n = 0

# Set up the initial potential.
phi = phi_initial.copy()
phi_n.x.array[:] = phi_initial.x.array

# Loop until completion.
while t < t_max:
    # Take snapshots for the video.
    phi_frame = phi_file % n
    phi_plotter.show(screenshot=phi_frame, auto_close=False) 
    
    # Update boundary conditions.
    t += dt.value
    rho.interpolate(rho_function(t))
    uD.interpolate(uD_function(t))

    # Assemble the RSH: the vector on the "right-hand side" of A.u = b.
    b.x.array[:] = 0
    fem.petsc.assemble_vector(b.vector, compiled_L)

    # Apply boundary condition.
    # These commands distribute the updated problem to all processes.
    fem.petsc.apply_lifting(b.vector, [compiled_a], [bcs])
    b.x.scatter_reverse(la.ScatterMode.add)
    fem.petsc.set_bc(b.vector, bcs)

    # Solve linear problem.
    solver.solve(b.vector, phi.vector)

    # Distribute the solution to all processes.
    phi.x.scatter_forward()

    # Update un --- the current value of the function.
    phi_n.x.array[:] = phi.x.array

    # Update the plots and save the frame.
    phi_grid.point_data["phi"] = phi.x.array.real
    phi_warped = phi_grid.warp_by_scalar("phi", factor=10)
    phi_plotter.update_scalars(phi.x.array.real, render=False)
    phi_plotter.update_coordinates(phi_warped.points, render=False)

    n += 1

# Take final snapshots for the video.
phi_frame = phi_file % n
phi_plotter.show(screenshot=phi_frame, auto_close=False)

# Close the plotter and finish processing the movie.
phi_plotter.close()

display.clear_output()

In [ ]:
!ffmpeg -y -i %03d-diffusion.jpg -pix_fmt yuv420p diffusion-movie.mp4

### Wave Equation

In [26]:
# Reset the initial potential.
phi = phi_initial.copy()
phi_n = fem.Function(V)
phi_n_1 = fem.Function(V)

In [27]:
# This is the problem we are going to solve:
# The wave equaiton, in weak form for finite elements.
u_n = 0.25*u + 0.5*phi_n + 0.25*phi_n_1
F = ufl.inner(u - 2*phi_n + phi_n_1, v) * ufl.dx
F += dt**2 * c**2 * ufl.inner(ufl.grad(u_n), ufl.grad(v)) * ufl.dx

# Add a diffusive term to keep the solution from blowing up right away.
F += c**2 * dt / D * ufl.inner(u - phi_n, v) * ufl.dx

# Add a potential source term.
F -= dt**2 * c**2 * ufl.inner(f, v) * ufl.dx

# Assemble the FEM problem.
(a, L) = ufl.system(F)

display.clear_output()

In [28]:
## Choose boundary conditions.
bcs = [fem.dirichletbc(uD, boundary_dofs)]
# bcs = []

In [29]:
# Building the matrix for the finite element problem takes a while.
# It does not change throughout the problem, either.  The following
# line build the matrix once and store it for reuse.
compiled_a = fem.form(a)
A = fem.petsc.assemble_matrix(compiled_a, bcs=bcs)
A.assemble()

display.clear_output()

In [30]:
# We do the same for the vector, but it will be updated at each time step.
compiled_L = fem.form(L)
b = fem.Function(V)

display.clear_output()

In [31]:
# Now we use the PETSc problem to construct the linear algebra problem.
solver = petsc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(petsc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(petsc.PC.Type.LU)

In [33]:
# Set potential to initial solution.
phi = phi_initial.copy()

# Create a plotter for charge density, but don't display the plot.
phi_file = "%03d-wave.jpg"

# It will be making a movie in the background.
phi_plotter = pyvista.Plotter(notebook=False, off_screen=True)

# And add the charge and potentials to the plot.
topology, cells, geometry = plot.create_vtk_mesh(phi.function_space)
phi_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
phi_grid.point_data["phi"] = phi.x.array.real
phi_grid.set_active_scalars("phi")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
phi_warped = phi_grid.warp_by_scalar("phi", factor=10)

# Add the warped grid to the plot.
phi_plotter.add_mesh(
    phi_warped,
    lighting=False,
    show_edges=False,
    clim=[0, 1],
    cmap='turbo'
);

ImportError: ignored

In [ ]:
# Get rid of old diffusion movie files.
!rm *-wave.jpg
!rm wave-movie.mp4

In [ ]:
# Now, compute an update for each time step and add it to the movie.
# Initial time
t = t0
n = 0

# Set up the initial potential.
phi = phi_initial.copy()
phi_n_1.x.array[:] = phi_initial.x.array
phi_n.x.array[:] = phi_initial.x.array


# Loop until completion.
while t < t_max:
    # Take snapshots for the video.
    phi_frame = phi_file % n
    phi_plotter.show(screenshot=phi_frame, auto_close=False) 
    
    # Update boundary conditions.
    t += dt.value
    rho.interpolate(rho_function(t))
    uD.interpolate(uD_function(t))

    # Assemble the RSH: the vector on the "right-hand side" of A.u = b.
    b.x.array[:] = 0
    fem.petsc.assemble_vector(b.vector, compiled_L)

    # Apply boundary condition.
    # These commands distribute the updated problem to all processes.
    fem.petsc.apply_lifting(b.vector, [compiled_a], [bcs])
    b.x.scatter_reverse(la.ScatterMode.add)
    fem.petsc.set_bc(b.vector, bcs)

    # Solve linear problem.
    solver.solve(b.vector, phi.vector)

    # Distribute the solution to all processes.
    phi.x.scatter_forward()

    # Update un --- the current value of the function.
    phi_n.x.array[:] = phi.x.array.real

    # Update the plots and save the frame.
    phi_grid.point_data["phi"] = phi.x.array.real
    phi_warped = phi_grid.warp_by_scalar("phi", factor=10)
    phi_plotter.update_scalars(phi.x.array.real, render=False)
    phi_plotter.update_coordinates(phi_warped.points, render=False)

    n += 1

# Take final snapshots for the video.
phi_frame = phi_file % n
phi_plotter.show(screenshot=phi_frame, auto_close=False)

# Close the plotter and finish processing the movie.
phi_plotter.close()

display.clear_output()

In [ ]:
!ffmpeg -y -i %03d-wave.jpg -pix_fmt yuv420p wave-movie.mp4

## Results

In [34]:
## Play the diffusion movie.
mp4 = open('diffusion-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

FileNotFoundError: ignored

In [35]:
# Play the wave movie.
mp4 = open('wave-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

FileNotFoundError: ignored

## Questions

Compare and contrast the two simulations.  Use the slider of the movie to view the action frame by frame.

- Describe similarities and differences between the diffusion simulation and the wave simulation.

***Describe your observations below.***

The simulations are not loading because of the same isues as the last notebook. However I did look up the difussion and waves and was able to learn a few things about them. Electric diffusion was said to be: "Diffusion current Density is a current in a semiconductor caused by the diffusion of charge carriers (electrons and/or electron holes). This is the current which is due to the transport of charges occurring because of non-uniform concentration of charged particles in a semiconductor." Basically diffusion is occuring because of non-uniform concentration of charged particles. So a electric wave on the other hand is refered to as: "A charge density wave (CDW) is an ordered quantum fluid of electrons in a linear chain compound or layered crystal. The electrons within a CDW form a standing wave pattern and sometimes collectively carry an electric current. The electrons in such a CDW, like those in a superconductor, can flow through a linear chain compound en masse, in a highly correlated fashion." I think this is highly useful information because a lot of technology can be built that uses this premise. Basicaly it is a wave of charged particles that are sent out. This is something I have actually always been curious about since the start of the term - if you could send a wave of charged particles. I am very glad I found my answer. This has been very rewarding.

# Helmholtz Equation

You probably noticed that the wave equation does not look right after a few frames.  This is due to ***numerical instability*** in the simulation.  Effects at the boundaries and processes that would take place "inside" one of our mesh cells or "between" our discrete time steps eventually cause the wave simulation to "blow up".

It's fun to watch, but it's not representative of what happens in a real system — most of the time.

There are techniques for dealing with this instability, but there are other ways to study the dynamics of waves, too.  A very important method is "plane wave decomposition" — a kind of Fourier analysis, where we study the behavior of waves of a single frequency.

We use separation of variables:
$$\phi(\vec{r},t) = \phi(\vec{r}) \, e^{-i\omega t}$$

In empty space, or in a uniform material, the spatial part of our solution can also be represented by a plane wave:
$$
\phi(\vec{r}) = A e^{i\vec{k} \cdot \vec{r}}
\qquad \implies \qquad
\phi(\vec{r},t) = A e^{i(\vec{k}\cdot\vec{r} - \omega t)}
$$
However, when the material is not uniform — an interface, an obstacle, an impurity — plane waves scatter.  The frequency does not change, but $\phi(\vec{r})$ can no longer be represented by a single wave vector $\vec{k}$.  We can still simplify the problem we have to solve, though.  If we substitute $\phi(\vec{r}) \, e^{-i\omega t}$ into the (generalized) wave equation, we find
\begin{align*}
0 &= \dfrac{1}{c^2} \dfrac{\partial^2 \phi}{\partial t^2} + \dfrac{1}{D} \dfrac{\partial \phi}{\partial t} - \nabla^2 \phi \\
&= \left(-\dfrac{\omega^2}{c^2} \phi(\vec{r})
- \dfrac{i\omega}{D} \phi(\vec{r}) - \nabla^2 \phi(\vec{r})\right)
e^{-i\omega t} \\
0 &= \nabla^2 \phi(\vec{r}) + \dfrac{\omega^2}{c^2} \phi(\vec{r})
+ \dfrac{i\omega}{D} \phi(\vec{r}) 
\end{align*}

This may not look like an improvement, but we now have a ***time-independent*** PDE for $\phi(\vec{r})$, which we can solve ***once*** to learn about the ***time-dependent*** behavior for ***all*** times!

This equation is similar to Laplace's equation and Poisson's equation, but not the same.  It is called ***the Helmholtz equation.***



## Model

Our original model of the "disappearing charge distribution" is not described by a single plane wave.

We will construct a new model instead: a plane wave that collides with an object that has a different index of refraction — hence, a different wave speed — than the surrounding medium.

In [36]:
# Set the frequency of the wave.
omega = 2 * np.pi

# Set the background index of refraction.
n0 = 1.0

# Set the maximum index of refraction inside the obstacle.
n_max = 1.0

# Set the size of the scattering object.
spread = 0.5

# Set the wave speed.
c = 1.0

# Compute the wave vector.
k0 = n0 * omega / c

## Geometry

In [37]:
# Use the same grid.
# Create a simple rectangular mesh.
length = 10
height = 10
Nx, Ny = 51, 51
extent = [[-length/2, -height/2], [length/2, height/2]]
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, extent, [Nx, Ny], mesh.CellType.triangle)

In [38]:
# Prepare the mesh for plotting.
topology, cells, geometry = dolfinx.plot.create_vtk_mesh(domain)

# Turn the mesh into a PyVista grid.
grid = pyvista.UnstructuredGrid(topology, cells, geometry)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# Save the HTML file.
plotter.export_html("./grid.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/grid.html')

## Obstacle

Add a dielectric object to the center of the region.  We'll describe it by a gaussian function.  You can picture it as concentric shells of decreasing dielectric strength.

In [39]:
# Define a set of functions on our mesh.
V = dolfinx.fem.FunctionSpace(domain, ("Lagrange", 2))

In [40]:
# Define the obstacle.
xC = 0
yC = 0
beta = 1/spread

# This function returns a function that can be interpolated.
def n_function(x0,y0):
    # return lambda x: n_0 + (n_max - n_0) * np.exp(-0.5 * beta**2 * ((x[0]-x0)**2 + (x[1] - y0)**2))
    return lambda x: n_0 + \
    (n_max - n_0) * np.exp(-0.5 * beta**2 * ((x[0]-x0)**2 + (x[1] + y0)**2)) + \
    (n_max - n_0) * np.exp(-0.5 * beta**2 * ((x[0]-x0)**2 + (x[1] - y0)**2))

# Turn this function definition into a time-dependent function on the mesh.
index = fem.Function(V)

# Initialize the function for t=0.
# index.interpolate(n_function(xC,yC))
index.interpolate(n_function(xC,2))

NameError: ignored

In [41]:
# Plot the index of refraction.
topology, cells, geometry = plot.create_vtk_mesh(index.function_space)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["n"] = index.x.array.real
grid.set_active_scalars("n")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
warped = grid.warp_by_scalar("n", factor=10)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Index of refraction"},
    clim=[n0, n_max],
    cmap='turbo'
)
plotter.add_text("Index of Refraction")
# Save the HTML file.
plotter.export_html("./index.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/index.html')

ImportError: ignored

## Boundary Conditions

The boundary conditions are quite different here.  We want a plane wave incident from the left (moving to the right).  We will fix the boundary conditions to represent this plane wave.  The other boundaries will be natural (open).

We need to match the value and the normal derivative of the incoming wave at the edges of the mesh.  To do this, we add a boundary term to the finite element problem:
$$\hat{n}\cdot\nabla u + in \dfrac{\omega}{c} u = \hat{n}\cdot\nabla\phi_{0} + in \dfrac{\omega}{c} \phi_{0}$$

In [42]:
n_hat = ufl.FacetNormal(domain)
x = ufl.SpatialCoordinate(domain)
phi0 = ufl.exp(1j * k0 * x[0])
g = ufl.dot(ufl.grad(phi0), n_hat) + 1j * k0 * phi0

display.clear_output()

## Finite Element Problem

In [43]:
# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
phi = fem.Function(V)

# This is the FEM version of the Helmholtz equation.
# It includes a boundary term, indicated by "ds".
a = - ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx \
    + index**2 * k0**2 * ufl.inner(u, v) * ufl.dx \
    + 1j * k0 * ufl.inner(u, v) * ufl.ds

# This is the boundary condition.
L = ufl.inner(g, v) * ufl.ds

# Put it all together for FEniCSx.
problem = fem.petsc.LinearProblem(a, L, u=phi, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# Now, solve it!
problem.solve()

# Tie up some loose ends.
phi.vector.ghostUpdate(addv=petsc.InsertMode.INSERT, mode=petsc.ScatterMode.FORWARD)

display.clear_output()

### Time Evolution

We can now simulate time evolution.  The spatial part of our solution does not change in time.  We need to multiply it by $e^{-i\omega t}$ and plot the real part as $t$ advances.

In [46]:
# Prepare the mesh for PyVista ...
topology, cells, geometry = plot.create_vtk_mesh(V)

# Create a plotter for charge density, but don't display the plot.
phi_file = "%03d-helmholtz.jpg"

# It will be making a movie in the background.
phi_plotter = pyvista.Plotter(notebook=False, off_screen=True)

# And add the charge and potentials to the plot.
topology, cells, geometry = plot.create_vtk_mesh(phi.function_space)
phi_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
phi_grid.point_data["phi"] = phi.x.array.real
phi_grid.set_active_scalars("phi")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
phi_warped = phi_grid.warp_by_scalar("phi", factor=0.2)

# Add the warped grid to the plot.
phi_plotter.add_mesh(
    phi_warped,
    lighting=False,
    show_edges=False,
    clim=[-2, 2],
    cmap='plasma'
);

ImportError: ignored

In [47]:
# Get rid of any old movie files.
!rm *-helmholtz.jpg
!rm helmholtz-movie.mp4

rm: cannot remove '*-helmholtz.jpg': No such file or directory
rm: cannot remove 'helmholtz-movie.mp4': No such file or directory


In [48]:
# Now, compute an update for each time step and add it to the movie.
# Initial time
t = 0.0
T = 2*np.pi / omega
t_max = 4 * T
dt = T / 60

n = 0

# Loop until completion.
while t < t_max:
    # Take snapshots for the video.
    phi_frame = phi_file % n
    phi_plotter.show(screenshot=phi_frame, auto_close=False) 
    
    # Update boundary conditions.
    t += dt
    
    # Update the plot and save the frame.
    phi_real = np.real(phi.x.array * np.exp(-1j * omega * t))
    phi_grid.point_data["phi"] = phi_real
    phi_warped = phi_grid.warp_by_scalar("phi", factor=0.2)
    phi_plotter.update_scalars(phi_real, render=False)
    phi_plotter.update_coordinates(phi_warped.points, render=False)
    phi_frame = phi_file % n

    n += 1

# Take final snapshots for the video.
phi_frame = phi_file % n
phi_plotter.show(screenshot=phi_frame, auto_close=False)

# Close the plotter and finish processing the movie.
phi_plotter.close()

In [49]:
# Assemble the movies with FFMPEG.
!ffmpeg -y -i %03d-helmholtz.jpg -pix_fmt yuv420p helmholtz-movie.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

## Results

Let look at the results.

In [50]:
# See the results.
topology, cells, geometry = plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["phi_R"] = phi.x.array.real
grid.point_data["phi_I"] = phi.x.array.imag
grid.point_data["phi_0"] = np.abs(phi.x.array)

In [51]:
# Plot the real part.
grid.set_active_scalars("phi_R")
warped = grid.warp_by_scalar("phi_R", factor=0.1)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Potential"},
    clim=[-2, 2],
    cmap='plasma'
)
plotter.add_text("Real Part")
# Save the HTML file.
plotter.export_html("./phi-real.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/phi-real.html')

ImportError: ignored

In [52]:
# Plot the imaginary part.
grid.set_active_scalars("phi_I")
warped = grid.warp_by_scalar("phi_I", factor=0.1)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Potential"},
    clim=[-2, 2],
    cmap='plasma'
)
plotter.add_text("Real Part")
# Save the HTML file.
plotter.export_html("./phi-imag.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/phi-imag.html')

ImportError: ignored

In [53]:
# Plot the modulus.
grid.set_active_scalars("phi_0")
warped = grid.warp_by_scalar("phi_0", factor=0.1)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Potential"},
    clim=[0, 2],
    cmap='plasma'
)
plotter.add_text("Real Part")
# Save the HTML file.
plotter.export_html("./phi-abs.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/phi-abs.html')

ImportError: ignored

In [54]:
# Play the potential movie.
mp4 = open('helmholtz-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Questions

Run the cells in the Helmholtz section of this notebook.  Inspect the plots and video that result.


### Plane wave

The index of refraction is constant on the entire mesh.

- Describe the plots you see.  What does the plane wave do in this case?
- Is the output consistent with what you expect?  Explain why or why not.

***Type your response below.***

The plots are not loading because of renderer and plotter code lines having issues however I can research and think about the question. The index of refraction - refracts whatever goes through it. A plane wave will be refracted to some angle after passing through the refraction medium.

### Adjust the frequency

Go to the "Model" section above (in the "Helmholtz Equation" section of the notebook).  You can change the frequency of the wave by adjusting $\omega$ — the `omega` variable.

- What do you expect to happen if you increase $\omega$?  If you decrease $\omega$?
- Double the value of $\omega$ or reduce it by half.  What happens in the plots and video?

***Include your predictions and observations below.***


Since we know that omega is the frequency of the wave then we know that if you modify omega it will modify the wavelength. So doubling omega will double the wavelength - half will half and decreasing will decrease. This is what causes the red shift or a blue shift. It is all like the doppler effect and it operates with light wavelengths as well as all electromagnetic frequencies.

### Introduce an obstacle

You can introduce an obstacle by changing the value of `n_max`.  This will introduce a round region of higher index of refraction in the center of the box.

- What do you expect to happen if you introduce such an obstacle?
- Set `n_max = 2.0`.  Describe what happens.

***Include your predictions and observations below.***

Adding an obstical will impead the flow of the wave. If we are introducing a round region of higher index of refraction in the center of the box then all the wave in the box will warp itself around the round region and since it is of a higher index of refraction it will be very strong to refracting the electromagnatic wave that makes contact to it.


### Exploration

Experiment with size, location, and index of refraction of the obstacle — and the frequency of the wave.

***Describe your experiments and observations below.***

All of the mentioned above - size - location - index of refraction - and the frequency of the wave will determine how it behaves. I can imagine that there will be parameters in which a larger paramater will balance out a smaller paramater and such.

## Bonus

Try one of the following, or your own experiments.

- Add one or more additional obstacles to the box.
- Create an obstacle with a sharp boundary instead of a smooth change in index of refraction.
- Add a flat interface with different indices of refraction on either side.
- Try to make a movie that illustrates Snell's law.
- Introduce a complex index of refraction in the obstacle.

***Describe your efforts and observations below.***

REPLACE WITH YOUR RESPONSE.

# Reflection and Summary

- What are the major takeaways of this assignment for you?
- What was the most difficult part of this assignment?
- What was the most interesting part of this assignment?
- What questions do you have?

There are a ton of takeaways from this assignment. I learned about Charge Density Waves (CDW) because I looked out charge density to learn about a problem in which the code was not working so that way I could still have a good answer to the question. That has answered a lot of questions to me about how electromagnetic waves work and a lot of theoretical applications that I could have for it in the future. I also took away how the index of refraction and other paramaters such as obsticals that effect a electromagnetic wave. It's really good to see how these things get effected because you begin to learn how to simulate these kinds of things in real life and that has a lot of scientific value.

The most difficult part of this assignment was the renderer and plotter code lines not working. That also was the case with the last assignment but it was much worse in the last assignment because of the noted errors at the top and throughout the notebook. Fortunetly I looked up and researched those topics when they came up when the code didn't work and I provided my best answer. I found this entire assignment very interesting because it has expanded my understanding of electromagnetic waves and also their equations. I do not have any questions at this time.